In [ ]:
import os

import numpy as np
from sklearn.utils import resample
from CML_tool.decorators import file_based_cacheing

In [ ]:
def fn_sign(size:tuple=(10,20,50)):
    
    return 

In [ ]:
# Define different distribution parameters
distributions = [
    {"mu": 0, "sigma": 1, "label": "Standard Normal"},
    {"mu": 2, "sigma": 0.5, "label": "Narrow Peak"},
    {"mu": -1, "sigma": 2, "label": "Wide Spread"},
    {"mu": 3, "sigma": 1.5, "label": "Shifted Right"}
]

num_samples = 100
num_bootstraps = 500

# Generate data matrix
matrix = np.column_stack([
        np.random.normal(dist['mu'], dist['sigma'], num_samples) 
        for dist in distributions
        ])

# Perform bootstrapping using sklearn's resample
boot_matrix = np.zeros((num_samples, len(distributions), num_bootstraps))
for i in range(num_bootstraps):
    # For each column (distribution), resample using sklearn
    for j in range(len(distributions)):
        boot_matrix[:, j, i] = resample(
            matrix[:, j], 
            n_samples=num_samples, 
            replace=True, 
            random_state=i  # Use iteration as random state for reproducibility
        )